# Eye of Emergency 🚨 - Analyse des Tweets de Catastrophes

## Objectif du projet
Développement d'un modèle d'apprentissage automatique capable de classer des tweets signalant des catastrophes naturelles réelles pour aider les intervenants d'urgence et le public à accéder à des informations précises et fiables en période de crise.

